In [9]:
# STEP 1: Install and import required libraries
!pip install folium geopy transformers torch pandas

import pandas as pd
import folium
from geopy.distance import geodesic
from transformers import pipeline

# STEP 2: Load your dataset (upload CSV to Colab first)
from google.colab import files
uploaded = files.upload()  # Upload the downloaded dataset

# Read the dataset
data = pd.read_csv("ev-charging-stations-india.csv")  # Change filename if needed
data.head()

Saving ev-charging-stations-india.csv to ev-charging-stations-india (2).csv


,name,state,city,address,lattitude,longitude,type
0,Neelkanth Star DC Charging Station,Haryana,Gurugram,"Neelkanth Star Karnal, NH 44, Gharunda, Kutail...",29.6019,76.9803,12.0
1,Galleria DC Charging Station,Haryana,Gurugram,"DLF Phase IV, Sector 28, Gurugram, Haryana 122022",28.4673,77.0818,12.0
2,Highway Xpress (Jaipur-Delhi) DC charging station,Rajasthan,Behror,"Jaipur to Delhi Road, Behror Midway, Behror, R...",27.8751,76.2760,12.0
3,Food Carnival DC Charging Station,Uttar Pradesh,Khatauli,"Fun and Food Carnival, NH 58, Khatauli Bypass,...",29.3105,77.7218,12.0
4,Food Carnival AC Charging Station,Uttar Pradesh,Khatauli,"NH 58, Khatauli Bypass, Bhainsi, Uttar Pradesh...",29.3105,77.7218,12.0


In [10]:
print(data.columns)


Index(['name', 'state', 'city', 'address', 'lattitude', 'longitude', 'type'], dtype='object')


In [11]:
import folium

# Try to automatically find latitude/longitude columns
lat_col = [col for col in data.columns if 'lat' in col.lower()][0]
lon_col = [col for col in data.columns if 'lon' in col.lower() or 'lng' in col.lower()][0]

# Try to find a name/ID column for popup
name_col = None
for possible in ['Station_Name', 'Station name', 'Name', 'Title']:
    if possible in data.columns:
        name_col = possible
        break

# Create base map of India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Add first 100 markers
for _, row in data.head(100).iterrows():
    folium.Marker(
        [row[lat_col], row[lon_col]],
        popup=row[name_col] if name_col else f"Lat: {row[lat_col]}, Lon: {row[lon_col]}",
        icon=folium.Icon(color="green", icon="bolt", prefix="fa")
    ).add_to(m)

m.save("ev_map.html")
m


In [12]:
from transformers import pipeline

# Load a better generative AI model (optional but smarter)
generator = pipeline("text-generation", model="gpt2-medium")

# Improved prompt with examples (helps model understand route format)
prompt = """You are an AI travel assistant that creates optimized electric vehicle routes.

Example 1:
From Delhi to Agra — Start with 90% battery, stop at Mathura EV Charger after 160 km, reach Agra with 35% charge.

Example 2:
From Bengaluru to Mysuru — Start with 85% battery, stop at Mandya Charging Station after 130 km, reach Mysuru with 40% battery.

Now suggest an optimized route:
From Chennai to Coimbatore — Start with 80% battery and include nearby charging stations.
"""

# Generate the route plan
result = generator(prompt, max_length=180, num_return_sequences=1, truncation=True)

print("⚡ AI Suggested Route:\n")
print(result[0]['generated_text'])


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=180) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


⚡ AI Suggested Route:

You are an AI travel assistant that creates optimized electric vehicle routes.

Example 1:
From Delhi to Agra — Start with 90% battery, stop at Mathura EV Charger after 160 km, reach Agra with 35% charge.

Example 2:
From Bengaluru to Mysuru — Start with 85% battery, stop at Mandya Charging Station after 130 km, reach Mysuru with 40% battery.

Now suggest an optimized route:
From Chennai to Coimbatore — Start with 80% battery and include nearby charging stations.

From Chennai to Delhi — Start with 90% battery and include nearby charging stations.

Example 3:

From Delhi to Hyderabad — Start with 80% battery and include nearby charging stations.

This is a small example but it shows that you can create an optimized route using Tesla's Powerwall battery pack.

Tesla's Powerwall is a modular solar panel that can power nearly a 100 kWh battery pack. It is available with a battery pack that can last up to three years, allowing you to recharge the battery pack in a fe